In [2]:
import openai
response = openai.ChatCompletion.create(


In [1]:
import os
import glob
import pickle
from datetime import datetime
import time
import dotenv
import pandas as pd

import requests
import requests.auth

import praw

import openai

from pydantic import BaseModel, Field, validator
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
openai.api_key = os.getenv('OPENAI_API_KEY')

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# load secrets from .env into environment variables
dotenv.load_dotenv()

praw.__version__

'7.7.0'

See README.md
 - objective is to use OpenAI for named entity extraction to extract all the songs form [this reddit thread](https://www.reddit.com/r/AskReddit/comments/12viv4v/what_is_the_prettiest_song_you_ever_heard_in_your/) and make Spotify playlist
 - use Reddit PRAW API to download all the comments (get [Reddit API key](https://www.reddit.com/prefs/apps))
 - use OpenAI API with a prompt like, extract all the songs from this text to CSV get ([OpenAI API key](https://platform.openai.com/account/api-keys))
 - use Spotify API to make a playlist (get [Spotify API key](https://developer.spotify.com/documentation/web-api/tutorials/getting-started))
 - works, needed a lot of scrubbing, but about 1 day of work, wouldn't have been possible to do a 700-song playlist manually without a team of Mechanical Turks or something
 - If I wanted to go nuts, would process comments individually, save a file for each comment's extracted songs, would make it easier to track down what OpenAI gets wrong, have a resumable, retryable, repeatable process and 
 - Spotify playist is [here](https://open.spotify.com/playlist/08YFkbtTV6GBfNtjJ4PHDu?si=f4761d983ac84091) 
 
 needs a .env file per dot-env-template
 

In [2]:
# a thread 
submission = "12viv4v"

# minimum karma to process a reply 
minkarma = 5

# a query to prompt a language model to populate the data structure.
prompt_template="""You will act as a research assistant finding all the artists and track titles in a document.
Define the JSON object schema as follows: 
{{"artist": "The Beatles", "track": "Yesterday"}}
{{"artist": "Eagles", "track": "Hotel California"}}

You will extract all artists and tracks from the following input, and return a list of records containing each artist and track extracted from the input in the JSON object schema. The input is:

{query}
"""

# a prompt to apply to replies on the thread
# prefix = """Define an example CSV file output as follows: 
# "artist","song_title"
# "The Beatles","Yesterday"
# "Eagles","Hotel California"

# Extract all song titles and artists from the following input, and return a CSV file output of the artists and song titles you extract from the input. If there were no songs extracted from the input, return "no songs found". the input is:
# """

# template="""You will act as a research assistant finding all the artists and track names in a document and returning them in a comma separated values format.

# {input_text}
# """
# system_prompt="You will act as a research assistant finding all the artists and track names in a document and returning them in a comma separated values format."
# assistant_prompt='Your response should be a list of comma separated values, eg: `"artist", "track_name"`'
# user_prefix="""You will extract all track names and artists from the input below.
# Your response should be a list of comma separated values containing the artists and track names you extract from the input. 
# The header row should contain `"artist","track_name"`. 
# The fields of each record should be enclosed in double-quotes. 
# The input is:
# """

# an output file to accumulate all the responses
savefile = 'bronze.txt'


## Get all comments from a reddit posting

In [3]:
def getPraw():
    return praw.Reddit(user_agent="prettiest_song/0.001", 
                       client_id=os.getenv('CLIENT_ID'), 
                       client_secret=os.getenv('CLIENT_SECRET'))


def getAll(r, submissionId, verbose=True):
    submission = r.submission(submissionId)
    submission.comments.replace_more(limit=None)
    commentsList=submission.comments.list()
    return commentsList


In [4]:
# print(datetime.now())
# r = getPraw()
# res = getAll(r, submission)
# print(datetime.now())

# print("retrieved ", len(res), 'comments')

In [5]:
# # we have a list of comment objects
# # filter comments with at least some karma
# res3 = [r for r in res if r.score >= minkarma]
# print('filtered to ', len(res3), 'comments')
# res3[0].body, res3[0].score

In [6]:
# # save so we can reload it later without downloading

# with open('reddit.pkl', 'wb') as f:
#     pickle.dump(res3, f)

In [7]:
    
with open('reddit.pkl', 'rb') as f:
    res3 = pickle.load(f)
    

## Extract artists and song titles using OpenAI

In [8]:
# check lengths of posts
shorties = []
big_ones = []
for i in range(len(res3)):
    if len(res3[i].body) <3:
        print (i, res3[i].body)
        shorties.append(i)
    if len(res3[i].body) > 4096:
        print(i, len(res3[i].body))
        big_ones.append(i)
        

474 4162
1539 W


In [9]:
# avg length
sum([len(r.body) for r in res3]) / len(res3)


105.34075546719681

In [10]:
test = res3[big_ones[0]].body
print (test[:500])

Saturn by Sleeping at Last:
https://www.youtube.com/watch?v=dzNvk80XY9s

The version they did with Tim Fain is even more beautiful: 
https://www.youtube.com/watch?v=0nRpeAiur9Q

I'm not good at choosing one thing from a list of favorites as the best, so I've got about 30+ answers that are really a 30+ -way tie, and the one that I would consider as "prettiest" at any given moment is heavily influenced by my current mood. So, it could be any one of these from my "Heart Wrenchingly Beautiful" playl


In [11]:
qa_model = 'gpt-3.5-turbo-0301'

temperature = 0.0
llm = ChatOpenAI(model_name=qa_model,
                 temperature=temperature)


In [12]:
# parser for output cleanup

# Define data structure.        
class ArtistTrack(BaseModel):
    artist: str = Field(description="name of an artist")
    track: str = Field(description="name of a track")

        
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=ArtistTrack)

# Cleanup parser can fix format if it doesn't work first time
cleanup_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)    


In [13]:
# define a chain using prompt template
llm_chain = LLMChain(
    prompt=PromptTemplate(
        template=prompt_template,
        input_variables=["query"],
    ),
    llm=llm
)

In [14]:
test = res3[5].body
test

'I fell in love with “Scarborough Fair” by Simon & Garfunkel as a kid and I still have a soft spot in my heart for it.\n\nEdit: Thanks for my first award(s)! I can’t wait to listen to all the covers of the song and other S&G songs you’ve suggested!'

In [15]:
print('calling llm')
output = llm_chain.run(test)
output = output.strip()
if output[0]=='[':
    output=output[1:]
if output[-1]==']':
    output=output[:-1]

calling llm


In [16]:
output_str = ''
err_str = ''
obj_list = []
for s in output.split("\n"):
        try:
            print('parser', s)
            output_obj = cleanup_parser.parse(s)
            obj_list.append(output_obj)
            output_str += str(output_obj) + "\n"
        except Exception as error1:
            print(s)
            print(error1)
            try:
                print('cleanup')
                output_obj = cleanup_parser.parse(s)
                obj_list.append(output_obj)
                output_str += str(output_obj) + "\n"
            except Exception as error2:
                print(error2)
                err_str += s + "\n"
                

parser {"artist": "Simon & Garfunkel", "track": "Scarborough Fair"}


In [64]:
# for each comment object we will extract the body 
# then submit as part of a prompt to chatgpt
print(datetime.now())

slist = res3.copy()

# to speed things we'll cumulate posts til we get to 100 posts or 5000 chars, whichever comes first
max_post_size=300  # redditor needs to put any songs in 1st couple hundred chars, c'mon
maxchars = 1000  # max tokens is 4096 but we'll limit each prompt to 2000 chars
# for json we seem to get timeouts from chatgpt on longer prompts
nposts = 100
total_posts = len(slist)
        
outdir = 'out'
logdir = 'logs'
# make sure out and logs are empty
for f in glob.glob('%s/*' % outdir):
    os.remove(f)
for f in glob.glob('%s/*' % logdir):
    os.remove(f)
count = 0
c = 0

obj_list = []  # store returned objects

while(slist):  # still comments to process
    input_text = ""
    reply_ids = []
    # build a string of next nposts replies or maxchars, whichever comes first
    for _ in range(nposts):  # add up to 100 posts to the prompt
        if slist:
            # make sure no single post > maxchars which breaks the logic below
            if len(slist[0].body) > max_post_size:
                slist[0].body = slist[0].body[:max_post_size]
            
            if len(input_text) + len(slist[0].body) < maxchars:
                reply = slist.pop(0)
                reply_ids.append(reply.id)
                body = reply.body # in order, for better context 
#                 if len(body) <3:
#                     print (c, body)
                input_text += body
                input_text += " \n \n"
                c += 1            

    # retry loop, have received untrapped 502 error
    output=''
    RETRIES = 3
    success = False
    for _ in range(RETRIES):
        try:
            # use langchain to try to get valid data every time
            # create prompt template > LLM chain
            output = llm_chain.run(input_text)
            output = output.strip()
            if output[0]=='[':
                output=output[1:]
            if output[-1]==']':
                output=output[:-1]
        except Exception as error:
            print("An exception occurred:", str(error))
            print("looping...")
            time.sleep(5)
            continue  # try again
        success = True
        break   # exception not triggered
    if not success:
        print('bailing')
        continue

            
    # parse output
#     print(output)
    output_str = ''
    err_str = ''
    for s in output.split("\n"):
#         print('=>', s)
        try:
            output_obj = cleanup_parser.parse(s)
            # success
            obj_list.append(output_obj)
            output_str += str(output_obj) + "\n"            
        except:
            err_str += s + "\n"
                
    # response = response['choices'][0]['message']['content']
    #     with open("%s/%04d.csv" % (outdir, count), 'w') as outfile:
    #         outfile.write(output_str)
    
    with open("%s/%04d.log" % (logdir, count), 'w') as logfile:
        logfile.write(str(reply_ids))
        logfile.write('\n\n===== raw prompt =====\n\n')        
        logfile.write(input_text)
        logfile.write('\n\n===== raw response =====\n\n')
        logfile.write(output)
        if err_str:
            logfile.write('\n\n===== failed validation =====\n\n')
            logfile.write("\n".join(err_str))
 
    count += 1
#     print(c)
    print(total_posts-len(slist), end=' ')

print()
print(datetime.now())



2023-05-04 11:30:37.629519
19 32 51 64 75 88 103 120 138 157 172 188 197 208 224 242 262 280 291 306 318 333 351 364 372 384 396 406 417 427 444 457 465 474 487 501 517 529 536 542 550 556 566 580 590 601 609 621 627 636 642 650 659 666 672 681 689 697 708 718 726 736 744 

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180).
Retr

An exception occurred: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180)
looping...
752 

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180).
Retr

An exception occurred: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180)
looping...


...

An exception occurred: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180)
looping...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180).
Retr

An exception occurred: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180)
looping...
759 767 772 778 786 795 802 812 820 828 837 846 854 863 870 884 893 900 905 909 917 928 941 960 963 972 982 994 1003 1015 1026 1038 1049 1057 1064 1075 1086 1095 1110 1115 1124 1137 1150 1156 1166 1180 1190 1199 1213 1222 1230 1236 1245 1253 1264 1277 1287 1298 1311 1322 1332 1346 1357 1365 1376 1389 1405 1414 1427 1435 1443 1452 1464 1475 1483 1493 1503 1517 1530 1544 1557 1564 1575 1583 1594 1606 1617 1625 1635 1643 1653 1658 1670 1673 1680 1693 1700 1706 1712 1721 1727 1739 1750 1760 1773 1780 1783 1790 1798 1805 1812 1819 1828 1835 1849 1858 1866 1874 1880 1890 1898 1911 1918 1931 1937 

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=180).
Retr

KeyboardInterrupt: 

In [25]:
artist_list = [o.artist for o in obj_list]
track_list = [o.track for o in obj_list]

df = pd.DataFrame({'artist': artist_list,
                   'track': track_list})

df.to_csv(savefile, index=False)

len(df)



2290

In [27]:
df = pd.read_csv(savefile) \
    .drop_duplicates() \
    .dropna() \
    .sort_values(["artist", "track"]) \
    .reset_index(drop=True)

df.to_csv('silver.csv', index=False)

print(len(df))
# tweak further to get to gold.csv

df


1312


,artist,track
0,311,Amber
1,A Perfect Circle,Blue
2,A Perfect Circle,By and Down the River
3,A Perfect Circle,Feathers
4,A Perfect Circle,Gravity
...,...,...
1307,unknown,The high notes in that piece are out of this w...
1308,unknown,Wreck of the Edmund Fitzgerald
1309,unknown,Yesterday
1310,unknown,played he ain’t heavy


In [29]:
df.groupby('artist') \
    .count() \
    .reset_index() \
    .sort_values('track', ascending=False) \
    .head(10)



,artist,track
569,Unknown,51
518,The Beatles,37
442,Radiohead,26
482,Simon & Garfunkel,24
179,Enya,16
271,Jim Croce,16
194,Fleet Foxes,14
420,Peter Gabriel,14
425,Pink Floyd,14
525,The Cure,14


In [36]:
df = df.drop(df.loc[df['artist'].str.lower()=='Unknown'].index)



In [35]:
df.groupby('track') \
    .count() \
    .reset_index() \
    .sort_values('artist', ascending=False) \
    .head(10)


,track,artist
102,Blackbird,5
347,Hallelujah,5
739,Scarborough Fair,5
126,Bridge Over Troubled Water,5
496,Landslide,4
920,Time,4
228,Dream a Little Dream of Me,4
118,Both Sides Now,4
164,Clair de Lune,4
351,Harvest Moon,3


In [34]:
df = df.drop(df.loc[df['track'].str.lower()=='unknown'].index)



In [37]:
len(df)


1216

In [38]:
df.to_csv('silver.csv', index=False)


## Load into a Spotify playlist


In [39]:
client_credentials_manager = SpotifyClientCredentials(client_id=os.getenv('SPOTIFY_CLIENT_ID'), 
                                                      client_secret=os.getenv('SPOTIFY_CLIENT_SECRET'),
                                                      )

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [41]:
# check artists
df = pd.read_csv("silver.csv")
df.drop_duplicates() \
    .dropna() \
    .sort_values(["artist", "track"])

dedupe = {}
fail_list = []

for index, artist, title in df.itertuples():
    if artist in dedupe:
        continue
    dedupe[artist]=1
    query_str = 'artist:%s' % (artist)
    artist_results = sp.search(q=query_str, type='artist', limit=3, offset=0, market='US')
    artist_names = [artist['name'] for artist in artist_results['artists']['items']]
    if artist_names:
        if artist.lower() != artist_names[0].lower():
            print(artist, artist_names)
    else:
        fail_list.append((artist, title))
        print("not found:", artist, "-", title)

# then clean up manually as appropriate

A-Ha ['Daryl Hall & John Oates', 'a-ha', 'half•alive']
not found: AJ Croce - Croce Plays Croce
Adeem ['Adeem the Artist', 'Adeem', 'Adeema']
Al Stewart ['Alexander Stewart', 'Al Stewart', 'Alec Lee-Stewart']
Alan Parsons Project ['The Alan Parsons Project', 'The Alan Parsons Symphonic Project', 'The Alan Parsons Tribute Project']
Albinoni ['Tomaso Albinoni', 'Albinoni Tommaso', "Tomaso Albinoni's Orchestra Trieste"]
not found: Allison Kraus - When you say nothing at all
not found: Allison Krauss and Brad Paisley - Whiskey Lullaby
Almost famous ['Original Broadway Cast of Almost Famous - The Musical', 'Almost Famous', 'Almost Famous Friends']
Alt J ['alt-J', 'Jose Torres El Rey De Alto Mando', 'Jeffrey Althouse']
America ['The All-American Rejects', 'America', 'American Authors']
Amy ['Amy Winehouse', 'Amy Lee', 'Amy Grant']
not found: Andrew Lloyd Webber, Sarah Brightman, Paul Miles-Kingston - Pie Jesu
Avril ['Avril Lavigne', 'Jane Avril', 'Lee Avril']
Babyface ['Babyface Ray', 'Babyfa

Smashing Pumpkins ['The Smashing Pumpkins']
Sophie B Hawkins ['Sophie B. Hawkins']
Sophie Hawkins ['Sophie B. Hawkins']
St Vincent ['St. Vincent', 'Vincent Steele', 'St. Vincent']
Sundays ['The Sundays', 'Sarah and the Sundays', 'Sundays By The Ocean']
Tchaikovsky ['Pyotr Ilyich Tchaikovsky', 'Tchaikovsky Chamber Orchestra', 'The Tchaikovsky Symphony Orchestra']
Terence Trent D’Arby ["Karaoke - Terence Trent D'arby"]
not found: Terrence Jay - One Blood
The Band ['The Band CAMINO', 'The Band', 'The Charlie Daniels Band']
The Dixie Chicks ['The Chicks']
The King’s Singers ["The King's Singers", 'The Kingdom Singers', 'The Bob King Singers']
The La’s ['The Kid LAROI', 'The Lacs', 'The Laurie Berkner Band']
The Mamas and the Papas ['The Mamas & The Papas', 'The Mamas & The Papas Experience', 'Karaoke - The Mamas & The Papas']
not found: Tiesto n Andain - Beautiful things
Various Artists ['Various Pro Artists', 'Various Ethiopian Artists', 'Tony Bennett with various duet artists']
Vince ['V

In [43]:
# check songs

df = pd.read_csv("silver.csv")
df.drop_duplicates() \
    .dropna() \
    .sort_values(["artist", "track"])

dedupe = {}
mylist = []
fail_list = []
artist_list, track_list, uri_list, album_list = [], [], [], []
orig_artist, orig_track = [], []

for index, artist, title in df.itertuples():
    query_str = 'artist:%s track:%s' % (artist, title)
    track_results = sp.search(q=query_str, type='track', limit=1, offset=0, market='US')
    results = track_results['tracks']['items']
    
    if results:
        r = results[0]
        # failsafe to never put same track twice
        if dedupe.get(r['id']):
            continue
        dedupe[r['id']]=True
        if title.lower() != r['name'].lower():
            print ("%s|%s : %s|%s" % (artist, title, r['artists'][0]['name'], r['name']))
        uri_list.append(r['uri'])
        artist_list.append(r['artists'][0]['name'])
        track_list.append(r['name'])
        album_list.append(r['album']['name'])
        orig_artist.append(artist)
        orig_track.append(title)
#         print('  ',
#               r['artists'][0]['name'],'|',
#               r['name'], '|',
#               r['album']['name'],'|',
#               r['album']['release_date'],'|',
#               r['popularity'])
    else:
        fail_list.append((artist, title))
        print("not found:", artist, "-", title)

A Perfect Circle|Three Libras : A Perfect Circle|3 Libras
not found: A Perfect Circle - Wings for Marie pt 1&2
not found: A-Ha - Take on Me (MTV unplugged version)
not found: AJ Croce - Croce Plays Croce
not found: Adeem - White Trash Revelry
not found: Adeem the Artist - The Sound
not found: Alan Walker - Cornfield Chase
Albinoni|Adagio : Tomaso Albinoni|Adagio in G Minor (Arr. for Harp and Orchestra)
not found: Alice In Chains - Don’t Follow
Alice in Chains|Down in a Hole : Alice In Chains|Down In A Hole (2022 Remaster)
not found: Alice in Chains - Mad Season's River of Deceit
Alice in Chains|Rain When i Die : Alice In Chains|Rain When I Die (2022 Remaster)
not found: Alison Krauss - Go rest high on that Mountain
not found: Allison Kraus - When you say nothing at all
not found: Allison Krauss and Brad Paisley - Whiskey Lullaby
not found: Almost famous - God damn that movies soundtrack is amazing
not found: Amy - a different melody of this song
not found: Anal Cunt - I Respect Your Fe

Enya|Lothlorien : Enya|Lothlórien
not found: Enya - all of her songs
Eric Clapton|Tears in heaven : Eric Clapton|Tears in Heaven - Acoustic Live
Erik Satie|Gnossienes : Erik Satie|Gnossienes: I. Lent
Erik Satie|Gymnopedie No.1 : Erik Satie|Gymnopédie No. 1
Erik Satie|Gymnopédie No. 3 : Erik Satie|3 Gymnopédies: No. 1 Lent et douloureux
not found: Etta James - I'd Rather Go Blind
not found: Etta James - I’d Rather Go Blind
Etta James|Sunday Kind of Love : Etta James|A Sunday Kind Of Love
not found: Eva Cassidy - All Thru the Night
Eva Cassidy|Somewhere Over the Rainbow : Karaoke - Eva Cassidy|Karaoke - Somewhere Over The Rainbow
not found: Eva Cassidy - any of her songs
not found: Evan Rachel Wood - Take these broken wings and learn to fly
Evanescence|Even In Death : Evanescence|Even In Death - 2016 Version
not found: Evanescence - True Colors
not found: Ewan McGregor - Come What May
not found: Ewan McGregor - Come what may
Ewan McGregor|Your Song : Ewan McGregor|Your Song - From "Mouli

Jon Bellion|Hand of God : Jon Bellion|Hand Of God - Outro
Joni Mitchell|Circle game : Joni Mitchell|The Circle Game
not found: Joni Mitchell - Roses Blue
Joni Mitchell|Blue : Joni Mitchell Tribute by Strings Attached|Blue Hotel Room
not found: Joni Mitchell - Help Me
Joni Mitchell|Urge for Going : Joni Mitchell|Mitchell: Urge for Going (Instrumental Arrangement of the B-Side Track of the Joni Mitchell Single "You Turn Me on I'm a Radio")
Josh Groban|Empty chairs : Josh Groban|Empty Chairs at Empty Tables (from "Les Misérables")
not found: Journey - Don't Stop Believin'
Joy Division|Shadowplay : Joy Division|Shadowplay - 2007 Remaster
not found: Jónsi - solo work
not found: Karen Carpenter - (They Long to Be) Close to You
not found: Karen Cowley - In A Week
not found: Kate Bush - Don’t Give Up
Kate Bush|This Woman's Work : Kate Bush|This Woman's Work - 2018 Remaster
Kate Bush|Wuthering Heights : Kate Bush|Wuthering Heights - 2018 Remaster
not found: Keith Moon - Yesterday
Lauryn Hill|Ex

Radiohead|Exit Music : Radiohead|Exit Music (For A Film)
not found: Radiohead - No alarms no surprises
not found: Radiohead - Plainsong
Radiohead|There There : Radiohead|There, There
Radiohead|Tinker Tailor Soldier Sailor... : Radiohead|Tinker Tailor Soldier Sailor Rich Man Poor Man Beggar Man Thief
Ray Charles|Georgia : Ray Charles|Georgia on My Mind
not found: Ray LaMontagne - Til the Sky Turns Black
not found: Regina Spektor - Sampson
not found: Reinbert de Leeuw - Gymnopédie No. 2
not found: Reinbert de Leeuw - Gymnopédie No. 3
not found: Ren - The Drugs Don’t Work
not found: Ricky Skaggs - Go rest high on that Mountain
Roberta Flack|First Time Ever I Saw Your Face : Roberta Flack|The First Time Ever I Saw Your Face
not found: Robin Pecknold - a very lonely solstice
Rod Stewart|Maggie : Rod Stewart|Maggie May
Saint-Saëns|The Swan : Camille Saint-Saëns|The Carnival of the Animals, R. 125: XIII. The Swan (Arr. for Cello and Piano)
Saint-Saëns|The Swan (Le Cygne) : Camille Saint-Saëns

The Postal Service|Such Great Heights : The Postal Service|Such Great Heights - Remastered
not found: The Postal Service - those songs
not found: The Pussycat Dolls - Killing Me Softly With His Song
The Rolling Stones|Wild Horses : The Rolling Stones|Wild Horses - 2009 Mix
not found: The Secret Sisters - cover of a song
The Shins|New Slang : The Shins|New Slang - 2021 Remaster
The Smashing Pumpkins|Mellon Collie and the Infinite Sadness : The Smashing Pumpkins|Mellon Collie And The Infinite Sadness - Remastered 2012
The Smiths|Please Please Please Let Me Get What I Want : The Smiths|Please, Please, Please, Let Me Get What I Want - 2011 Remaster
not found: The Smiths - Stop me if you think that you’ve heard this one before
The Smiths|That Joke Isn’t Funny Anymore : The Smiths|That Joke Isn't Funny Anymore
The Smiths|The Light That Never Goes Out : The Smiths|There Is a Light That Never Goes Out - 2011 Remaster
not found: The Smiths - museum track in Ferris Buellers Day Off
not found: Th

In [44]:
gold_df = pd.DataFrame({'input_artist': orig_artist,
                        'artist': artist_list,
                        'input_track': orig_track,
                        'track': track_list,
                        'album': album_list,
                        'uri': uri_list})
gold_df

,input_artist,artist,input_track,track,album,uri
0,311,311,Amber,Amber,Greatest Hits '93 - '03,spotify:track:51UtgWS4z1eMPuLQOzPtNH
1,A Perfect Circle,A Perfect Circle,Blue,Blue,Thirteenth Step,spotify:track:6tgTTBaIf0tO6lvDhoXfMg
2,A Perfect Circle,A Perfect Circle,By and Down the River,By And Down The River,Eat The Elephant,spotify:track:1gv7dUAsupITkccwEqkJqM
3,A Perfect Circle,A Perfect Circle,Feathers,Feathers,Eat The Elephant,spotify:track:26aVyDKN3MDAMoyO3VDB3g
4,A Perfect Circle,A Perfect Circle,Gravity,Gravity,Thirteenth Step,spotify:track:1CO4BB8CaiQggtJ0R6GwGt
...,...,...,...,...,...,...
867,Evanescence,Evanescence,Good enough,Good Enough,The Open Door,spotify:track:0eHxqgbpr2ptXQtw8Ab3ve
868,Grimes,Grimes,Symphonia IX,Symphonia IX (My Wait Is U),Visions,spotify:track:76iqrqfj4k7Igq2WoKuTlS
869,Machineheart,machineheart,Stonecold,Stonecold,Stonecold,spotify:track:5SmYtofSMKeccnqWksS9vW
870,Moby,Moby,Porcelain,Porcelain,Play & Play: B Sides,spotify:track:5fJAYWoFgXm26HLNViFiUh


In [45]:
with pd.option_context("display.max_rows", 999):
    display(gold_df.loc[gold_df['input_artist'].str.lower() != gold_df['artist'].str.lower()])
    
    

,input_artist,artist,input_track,track,album,uri
17,Alan Parsons Project,The Alan Parsons Project,Old and Wise,Old and Wise,Eye In The Sky (Expanded Edition),spotify:track:5Jt2AQv1c3RUF5ENtAYF1i
18,Alan Parsons Project,The Alan Parsons Project,Silence and I,Silence and I,Eye In The Sky (Expanded Edition),spotify:track:1MXd1awM0A7T7FxUPKAujf
19,Alan Parsons Project,The Alan Parsons Project,Time,Time,The Turn Of A Friendly Card (Expanded Edition),spotify:track:48yJZwYYDZX5GKFND7wDfC
21,Albinoni,Tomaso Albinoni,Adagio,Adagio in G Minor (Arr. for Harp and Orchestra),Notte Veneziana,spotify:track:47xdo9qYjAbOIVVBAqom1b
30,Alt J,alt-J,Dissolve Me,Dissolve Me,An Awesome Wave,spotify:track:2Dv7PTwSoB17f3VFDIKw8m
39,"Andrew Lloyd Webber, Sarah Brightman, Paul Mil...",Andrew Lloyd Webber,Pie Jesu,Pie Jesu,Diva: The Singles Collection,spotify:track:4D2Ha1TUtxuDqHk2Kb9DO4
51,Art Garfunkel,Art Garfunkel jr.,Bridge Over Troubled Water,Geh mit mir durch den Regenbogen (Bridge Over ...,Wie Du: Hommage an meinen Vater (Zweite Edition),spotify:track:0WxC7uOKJh0Dcn2by2onot
59,Barber,Samuel Barber,Adagio for Strings,Barber: Adagio for Strings,Samuel Barber - Adagio,spotify:track:1CSaCKPIp2yCIDL3t7Fyau
60,Barber,Samuel Barber,Agnus Dei,Agnus Dei,Enchanted Isle,spotify:track:02vw0tjLamMJAzMlCSiNH3
63,Beach Boys,The Beach Boys,God only knows,God Only Knows - Mono,Pet Sounds (Original Mono & Stereo Mix),spotify:track:6iGU74CwXuT4XVepjc9Emf


In [46]:
with pd.option_context("display.max_rows", 999):
    display(gold_df.loc[gold_df['input_track'].str.lower() != gold_df['track'].str.lower()])
    
    

,input_artist,artist,input_track,track,album,uri
8,A Perfect Circle,A Perfect Circle,Three Libras,3 Libras,Mer De Noms,spotify:track:5kHkaBN8OEQlmXfQkACxSt
21,Albinoni,Tomaso Albinoni,Adagio,Adagio in G Minor (Arr. for Harp and Orchestra),Notte Veneziana,spotify:track:47xdo9qYjAbOIVVBAqom1b
23,Alice in Chains,Alice In Chains,Down in a Hole,Down In A Hole (2022 Remaster),Dirt (2022 Remaster),spotify:track:7FRfYOql61DGDp9VPPe2qA
25,Alice in Chains,Alice In Chains,Rain When i Die,Rain When I Die (2022 Remaster),Dirt (2022 Remaster),spotify:track:6a9SPVrXyrlVh5Fh08f8Bz
38,Andrea Bocelli,Andrea Bocelli,Con te partiro,Con te partirò,Bocelli (Remastered),spotify:track:7zrpoAJte9o12TzawqgdD0
40,Andy Williams,Andy Williams,Moon River,"Moon River (From ""Breakfast at Tiffany's"")",Moon River And Other Great Movie Themes,spotify:track:24AIahNHzBxm9S12peXbnG
41,Anne Murray,Anne Murray,Danny song,Danny's Song,Danny's Song,spotify:track:5lQxMlCMTzCj3j4vyzdl8T
44,Antonio Vivaldi,Antonio Vivaldi,The Four Seasons: Winter,"The Four Seasons - Winter in F Minor, RV. 297:...",The Four Seasons & Concertos for Bassoon and V...,spotify:track:0ROMalDdNg3L2HoZP4qFyJ
46,Aphex Twin,Aphex Twin,Avril 14,Avril 14th,Drukqs,spotify:track:1uaGSDFsLdReQgg8p7Obwh
48,Aphex Twin,Aphex Twin,IZ Us,IZ-US,Come To Daddy,spotify:track:6INSqTqDoz6ndc0oKTi0Sq


In [47]:
gold_df2=gold_df.copy().reset_index(drop=True)
gold_df2['input_track']=gold_df2['input_track'].str.lower()
gold_df2['input_track']=gold_df2['input_track'].apply(lambda s: s.strip()[:10])

gold_df2['track']=gold_df2['track'].str.lower()
gold_df2['track']=gold_df2['track'].apply(lambda s: s.strip()[:10])


with pd.option_context("display.max_rows", 999):
    display(gold_df2.loc[gold_df2['input_track'] != gold_df2['track']])


,input_artist,artist,input_track,track,album,uri
8,A Perfect Circle,A Perfect Circle,three libr,3 libras,Mer De Noms,spotify:track:5kHkaBN8OEQlmXfQkACxSt
21,Albinoni,Tomaso Albinoni,adagio,adagio in,Notte Veneziana,spotify:track:47xdo9qYjAbOIVVBAqom1b
41,Anne Murray,Anne Murray,danny song,danny's so,Danny's Song,spotify:track:5lQxMlCMTzCj3j4vyzdl8T
46,Aphex Twin,Aphex Twin,avril 14,avril 14th,Drukqs,spotify:track:1uaGSDFsLdReQgg8p7Obwh
48,Aphex Twin,Aphex Twin,iz us,iz-us,Come To Daddy,spotify:track:6INSqTqDoz6ndc0oKTi0Sq
49,Aphex Twin,Aphex Twin,rhubarb,donkey rhu,Donkey Rhubarb,spotify:track:3IfmatoAjPtQOqteemdUnp
51,Art Garfunkel,Art Garfunkel jr.,bridge ove,geh mit mi,Wie Du: Hommage an meinen Vater (Zweite Edition),spotify:track:0WxC7uOKJh0Dcn2by2onot
59,Barber,Samuel Barber,adagio for,barber: ad,Samuel Barber - Adagio,spotify:track:1CSaCKPIp2yCIDL3t7Fyau
61,Basil Poledouris,Basil Poledouris,theology/c,conan the,Conan the Barbarian (Arr. P. Pelster),spotify:track:2K6NHYgiXgMsP27zjhxaaT
69,Beethoven,Ludwig van Beethoven,fur elise,bagatelle,"Für Elise, Bagatelle No. 25 in A Minor, WoO 59",spotify:track:3zLTPuucd3e6TxZnu2dlVS


In [ ]:
# these are songs that look like covers or otherwise not the expected response from spotify search 
# (which is a bit wonky, doesn't like quotes and such)

bad_lookups = [
    28, 79, 110, 286, 675, 693, 696, 697, 698, 699, 721, 726, 754
]

for i in bad_lookups:
    print(gold_df.iloc[i])
    
# add manually, plus 'not found'


In [ ]:
gold_df = gold_df.drop(
    axis='index',
    labels=bad_lookups)

gold_df[['artist', 'track']].to_csv('gold.csv', index=False)

with pd.option_context("display.max_rows", 999):
    display(gold_df)

In [48]:
# get playlist id
# first create a playlist in UI to load songs
playlists = sp.user_playlists(os.getenv('SPOTIFY_USERNAME'))
while playlists:
    for i, playlist in enumerate(playlists['items']):
        if playlist['name'] != 'Reddit Prettiest Songs':
            continue
        print(playlist['id'])
        playlist_id = playlist['id']
        print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

08YFkbtTV6GBfNtjJ4PHDu
   1 spotify:playlist:08YFkbtTV6GBfNtjJ4PHDu Reddit Prettiest Songs


In [49]:
# must follow an oauth workflow to write a playlist in Spotify
# running this cell should request a spotify login and then redirect to an url
# paste whole url with id into form to authenticate

scope = "playlist-modify-public"

sp = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth(scope=scope,
                                                       client_id=os.getenv('SPOTIFY_CLIENT_ID'),
                                                       client_secret=os.getenv('SPOTIFY_CLIENT_SECRET'),
                                                       redirect_uri="https://druce.ai"
                                                      ))

In [ ]:
addlist = gold_df['uri'].to_list()
print (len(addlist))

while(addlist):
    sp.user_playlist_add_tracks(os.getenv('SPOTIFY_USERNAME'), 
                                playlist_id=playlist_id, 
                                tracks=addlist[-100:])
    addlist = addlist[:-100]
    print("added items, remaining ", len(addlist))
    


In [ ]:
# manually add the ones that weren't found for some reason


In [56]:
# can run again and add any new tracks, either because OpenAI is a bit random, or new replies in thread
results = sp.user_playlist(os.getenv('SPOTIFY_USERNAME'), playlist_id,
                                fields='tracks,next,name')
tracks = results['tracks']

playlist_dict_by_uri = {}
playlist_dict_by_str = {}

artist_list = []
track_list = []
uri_list = []
popularity_list = []
album_list=[]

while True:
    for track_item in tracks['items']:
        track_dict = track_item['track']
        track_str = track_dict['artists'][0]['name']  + ' | ' + track_dict['name'][:15]
        uri = track_dict['uri']
        if track_str in playlist_dict_by_str:
            print(track_str)
        playlist_dict_by_str[track_str] = uri
        playlist_dict_by_uri[uri] = track_str
        
        uri_list.append(uri)
        artist_list.append(track_dict['artists'][0]['name'])
        track_list.append(track_dict['name'])
        album_list.append(track_dict['album']['name'])
        popularity_list.append(track_dict['popularity'])
        
    # check if there are more pages
    if tracks['next']:
        tracks = sp.next(tracks)
    else:
        break

print (len(list(playlist_dict_by_str.keys())))
print (len(list(playlist_dict_by_uri.keys())))


Nine Inch Nails | A Warm Place
Joni Mitchell | Both Sides Now
1028
1030


In [57]:
playlist_df = pd.DataFrame({'artist': artist_list,
                           'track': track_list,
                           'album': album_list,
                           'popularity': popularity_list,
                           })



In [58]:
with pd.option_context("display.max_rows", 9999):
    display(playlist_df.sort_values('popularity'))
    

,artist,track,album,popularity
923,Yusuf / Cat Stevens,Wild World,The Best Of Cat Stevens 20th Century Masters T...,0
779,Jaakko Aukusti,What If All Else Fails?,What If All Else Fails?,0
548,Edvard Grieg,"Peer Gynt, Op. 23, Act IV: Morning Mood",TikTok Klassische Musik,0
1017,Joni Mitchell,Both Sides Now,Clouds,0
1026,Johann Sebastian Bach,"Orchestral Suite No. 3 in D Major, BWV 1068: I...",Classical Music In the Background,1
305,Jay-Jay Johanson,Poison,Poison,1
1016,Joni Mitchell,Both Sides Now,Both Sides Now,1
42,The Smiths,Asleep,mental health: look out for yourself,2
31,The Niro,No One Must Find You Here,The Complete Jeff Buckley and Gary Lucas Songbook,2
371,Ennio Morricone,"""Finale"" from Once Upon a Time in the West - I...",Film Music,4


In [59]:
gold_dict_by_uri = {}
gold_dict_by_str = {}
addlist = []
c = 0
for i, artist, track, uri in gold_df[['artist', 'track', 'uri']].itertuples():
    # print(artist, track, uri)
    track_str = artist + ' | ' + track[:15]
    if track_str not in playlist_dict_by_str:
        addlist.append([artist, track, uri])
        print(artist, track, uri)
    gold_dict_by_uri[uri]=track_str
    gold_dict_by_str['track_str']= uri
#     if track_str not in playlist_dict_by_str:
#         c += 1
#         print (c, track_str)
        
print(len(gold_dict_by_str.items()))
print(len(gold_dict_by_uri.items()))

ABBA Dancing Queen spotify:track:0GjEhVFGZW8afUYGChu3Rr
ABBA One Of Us spotify:track:6zgtBUEkAfilJ2YEOvNexR
Adam Sandler Grow Old with You spotify:track:2lVYZep52w1efoH14yNRHg
Adele Hello spotify:track:62PaSfnXSMyLshYJrlTuL3
Alice In Chains I Stay Away spotify:track:6HnZsLUDrG3HTSbh8sq2u3
Alison Krauss Baby Mine spotify:track:7GJ8aXAGdfjo2OAh6XreB6
Amy Ray Blender spotify:track:6qJ8qGNSfBcVkD64qBEvfg
Art Garfunkel jr. Geh mit mir durch den Regenbogen (Bridge Over Troubled Water) spotify:track:0WxC7uOKJh0Dcn2by2onot
Avril Lavigne Complicated spotify:track:5xEM5hIgJ1jjgcEBfpkt2F
Barbara Ma plus belle histoire d'amour spotify:track:0qBVET4VkHsQAoboWlQ2pJ
Samuel Barber Agnus Dei spotify:track:02vw0tjLamMJAzMlCSiNH3
Bernie Taupin Your Song spotify:track:5BCsFeUrzPBXuSRUrbP6YP
Billy Joel The River of Dreams spotify:track:30qVCFYKBtAENjTIBA8FPZ
Björkliden Yoga Nidra spotify:track:3AdIPVFDkqSSvBjKFcN0mr
Bob Dylan Like a Rolling Stone spotify:track:3AhXZa8sUQht0UEdBJgpGc
Bob Dylan Mississippi -

In [60]:
addlist



[['ABBA', 'Dancing Queen', 'spotify:track:0GjEhVFGZW8afUYGChu3Rr'],
 ['ABBA', 'One Of Us', 'spotify:track:6zgtBUEkAfilJ2YEOvNexR'],
 ['Adam Sandler', 'Grow Old with You', 'spotify:track:2lVYZep52w1efoH14yNRHg'],
 ['Adele', 'Hello', 'spotify:track:62PaSfnXSMyLshYJrlTuL3'],
 ['Alice In Chains', 'I Stay Away', 'spotify:track:6HnZsLUDrG3HTSbh8sq2u3'],
 ['Alison Krauss', 'Baby Mine', 'spotify:track:7GJ8aXAGdfjo2OAh6XreB6'],
 ['Amy Ray', 'Blender', 'spotify:track:6qJ8qGNSfBcVkD64qBEvfg'],
 ['Art Garfunkel jr.',
  'Geh mit mir durch den Regenbogen (Bridge Over Troubled Water)',
  'spotify:track:0WxC7uOKJh0Dcn2by2onot'],
 ['Avril Lavigne', 'Complicated', 'spotify:track:5xEM5hIgJ1jjgcEBfpkt2F'],
 ['Barbara',
  "Ma plus belle histoire d'amour",
  'spotify:track:0qBVET4VkHsQAoboWlQ2pJ'],
 ['Samuel Barber', 'Agnus Dei', 'spotify:track:02vw0tjLamMJAzMlCSiNH3'],
 ['Bernie Taupin', 'Your Song', 'spotify:track:5BCsFeUrzPBXuSRUrbP6YP'],
 ['Billy Joel', 'The River of Dreams', 'spotify:track:30qVCFYKBtAE

In [ ]:
 
 ['Barbara',  "Ma plus belle histoire d'amour",
  'spotify:track:0qBVET4VkHsQAoboWlQ2pJ'],
 ['Samuel Barber', 'Agnus Dei', 'spotify:track:02vw0tjLamMJAzMlCSiNH3'],
 ['Bernie Taupin', 'Your Song', 'spotify:track:5BCsFeUrzPBXuSRUrbP6YP'],
 ['Billy Joel', 'The River of Dreams', 'spotify:track:30qVCFYKBtAENjTIBA8FPZ'],
 ['Björkliden', 'Yoga Nidra', 'spotify:track:3AdIPVFDkqSSvBjKFcN0mr'],
 ['Bob Dylan', 'Like a Rolling Stone', 'spotify:track:3AhXZa8sUQht0UEdBJgpGc'],
 ['Bob Dylan',  'Mississippi - Version 2',  'spotify:track:6JWHNd8QMxTvojYkmZtKGI'],
 ['Lil Bowser',  'I Stole Peaches From Mario!',  'spotify:track:601n7pnpUEebOo0SAl4lHt'],
 ['Bread', 'Diary', 'spotify:track:0VdcSHXCPRfQQZYQPXSlZ9'],
 ['Brian Chase', 'Well Drums', 'spotify:track:0Ym6ReMfwgKMga3XoJ7a1y'],
 ['Bright Eyes',  'First Day of My Life',  'spotify:track:0eBryM7ePQH3Klt3jz8xZd'],
 ['Cannibal Corpse', 'I Cum Blood', 'spotify:track:3bQoMbPYOvvseHaSOWAQ3Q'],
 ['Cass Elliot',  'Dream A Little Dream Of Me',  'spotify:track:3cOWCcBKwiwUs5PCYirwMI'],
 ['Christopher',  'Hope This Song Is For You (From the Netflix Film ‘A Beautiful Life’)',  'spotify:track:6hfd87ZTyziYGNdA7HYMv9'],
 ['Chris Cornell', 'Billie Jean', 'spotify:track:7sZD8tOmfRUObogGX8VzfZ'],
 ['Chris Cornell',  'Black Hole Sun - Recorded Live At Red Robinson Show Theatre, Vancouver, Canada on April 30, 2011',  'spotify:track:5p0s4na5wJHnaNw9azmiQm'],
 ['Cigarettes After Sex',  'Apocalypse',  'spotify:track:3AVrVz5rK8Hrqo9YGiVGN5'],
 ['Claude Debussy', 'Claire de lune', 'spotify:track:6kf7ZCJjEbjZXikivKOsvJ'],
 ['Cocteau Twins',  'Pearly-Dewdrops\' Drops - 7" Version',  'spotify:track:0mxe1uof85IOstFsn2l2da'],
 ['Cocteau Twins',  'The Spangle Maker',  'spotify:track:0S91YiVAlTJ25AAaCjKgdS'],
 ['Cocteau Twins',  'Those Eyes, That Mouth',  'spotify:track:67FMvER1wXDIGzManwB4Xk'],
 ['Creed', 'My Sacrifice', 'spotify:track:2VSbEXqs6NbNiZSTcHlIDR'],
 ['Creedence Clearwater Revival',  'Bad Moon Rising',  'spotify:track:20OFwXhEXf12DzwXmaV7fj'],
 ['Crowded House',  'Don’t Dream It’s Over - Home Demo',  'spotify:track:0fiSpF9mvRFQWy0ca64d1g'],
 ['Claude Debussy',  'Deux Arabesques (Two Arabesques), L. 66: No. 1 in E Major',  'spotify:track:2iUeGXoh8cDXVBp7QnTRCT'],
 ['Léo Delibes', 'Flower Duet', 'spotify:track:5K8jqeLAxZIqHR6e5w5so1'],
 ['Desiree Weekes',  'Kissing You (arr. J. Wikeley for choir)',  'spotify:track:242Uewkch7cnpgn9IHDyMz'],
 ['Dire Straits', 'Brothers In Arms', 'spotify:track:7itoreKcEK6xfGQx4nY4Rh'],
 ['The Chicks',  'Not Ready to Make Nice',  'spotify:track:6LCEyZZHFF4ebF1Mike1s5'],
 ['Don McLean',  'Vincent (Starry, Starry Night)',  'spotify:track:2YDyH60Vro33KkDtNZCXIk'],
 ['Elvis Presley',  'Can’t Help Falling in Love - Acoustic Cover',  'spotify:track:0ghQkNDYLSl4GsqfkjTjWx'],
 ['Enya', 'Amarantine', 'spotify:track:0VmzazQQ0Mo1vJldr5NxTW'],
 ['Erik Satie', 'Gnossienes: I. Lent', 'spotify:track:6KLL2PtvBHR5XXDlvioVpy'],
 ['Etta James',  'A Sunday Kind Of Love',  'spotify:track:0zGLlXbHlrAyBN1x6sY0rb'],
 ['Karaoke - Eva Cassidy',  'Karaoke - Somewhere Over The Rainbow',  'spotify:track:7a6lTqj1NQvNBtpyDyu27U'],
 ['Fleet Foxes', 'English House', 'spotify:track:4dv5EEuSVwCBtayhHUUlKX'],
 ['Fleetwood Mac', 'Rhiannon', 'spotify:track:05oETzWbd4SI33qK2gbJfR'],
 ['Frank Ocean', 'Thinkin Bout You', 'spotify:track:7DfFc7a6Rwfi3YQMRbDMau'],
 ['Frank Sinatra', 'My Way', 'spotify:track:3spdoTYpuCpmq19tuD0bOe'],
 ['Frank Zappa', "Joe's Garage", 'spotify:track:2HudfMAHpahXeWpgeZkk3r'],
 ['Art Garfunkel', 'Watermark', 'spotify:track:3LE247gGqUCFxci5BwfXmv'],
 ['Brandon Garth', 'The Dance', 'spotify:track:08MhzVza8Qc4M172KZ37JT'],
 ['George Harrison',  'While My Guitar Gently Weeps - Live At Madison Square Garden; 2009 Remaster',  'spotify:track:4Egi6XuC0rbLlXfqmQeuFa'],
 ['George Frideric Handel',  'Solomon, HWV 67: Will the sun forget to streak (Arr. for Piano)',  'spotify:track:1t5vEDV8nP9vEG4SuJ096b'],
 ['Glenn Miller', 'In the Mood', 'spotify:track:1xsY8IFXUrxeet1Fcmk4oC'],
 ['The Goo Goo Dolls', 'Acoustic #3', 'spotify:track:1RC9slv335IfLce5vt9KTW'],
 ['Gordon Lightfoot',  "Song for a Winter's Night",
  'spotify:track:6R6YmDz2W8A2irGDVUnrSh'],
 ['Gordon Lightfoot', 'Sundown', 'spotify:track:0SjnBEHZVXgCKvOrpvzL2k'],
 ['Gorillaz', 'Feel Good Inc.', 'spotify:track:0d28khcov6AiegSCpG5TuT'],
 ['H.E.R.', 'Damage', 'spotify:track:0KS2h61pHQ4WmOwruD7uxD'],
 ['Hans Zimmer',  'Day One (Interstellar Theme)',  'spotify:track:4WmB04GBqS4xPMYN9dHgBw'],
 ['Harold Arlen',  'The Wizard of Oz: Somewhere Over The Rainbow',  'spotify:track:1dq9JjhCNTonsObgjYK8Px'],
 ['Hellbound Hearts', "Nancy's House", 'spotify:track:4tDlkhY0hD6hOL4RsOK5hC'],
 ['Hozier', 'Take Me to Church', 'spotify:track:1CS7Sd1u5tWkstBhpssyjP'],
 ['Iron & Wine', 'Woman King', 'spotify:track:2JBZPY3dLQJy529XJnljKu'],
 ['Jack Black', 'Peaches', 'spotify:track:4w9soAM7IrmYDhSXLp14p6'],
 ['James Taylor',  'You’ve Got a Friend',  'spotify:track:3nK4hWsTEr7fVXziI5bTmh'],
 ['Jeff Buckley',  'Calling You - Live at Sin-é, New York, NY - July/August 1993',  'spotify:track:76n5xDPCuw0bBY38s3kWYD'],
 ['Jeff Buckley',  'Everybody Here Wants You',  'spotify:track:2bcvooA6HEmVUneEGJnNZD'],
 ['Jeff Buckley', 'Last Goodbye', 'spotify:track:0y5CnV2idm2KkQEudDjfDT'],
 ['Jeff Buckley',  "Je N'en Connais Pas La Fin - Live at Sin-é, New York, NY - July/August 1993",
  'spotify:track:4mO7DeDHWeyU8RVII7UWwm'],
 ['Jimi Hendrix', 'One Rainy Wish', 'spotify:track:5Zyv0v4rPcrXjkaeImuodv'],
 ['Jimi Hendrix',  'Spanish Castle Magic',  'spotify:track:2KFE98Iw0X23sf4vJYcbLH'],
 ['Jimi Hendrix',  'Wait Until Tomorrow',  'spotify:track:2YtVzmZzew1ILUdNueyWd7'],
 ['John Coltrane',  'Giant Steps - 2020 Remaster',  'spotify:track:1ZXu0ib26kWfQQngREMcU2'],
 ['Johnny Doe', "My Mother's Bible", 'spotify:track:6b7VGKI8gsLHXWys9mw7hD'],
 ['John Lennon',  'Imagine - Remastered 2010',  'spotify:track:7pKfPomDEeI4TPT6EOYjn9'],
 ['John Rutter',  'Rutter: Requiem: Pie Jesu',  'spotify:track:48x7KPjY5apjU369NrXDlQ'],
 ['Johnny Cash',  "I Don't Know Where I'm Bound - Live at San Quentin State Prison, San Quentin, CA - February 1969",
  'spotify:track:3JIC4rykTBTiIobbqHY11j'],
 ['Joni Mitchell Tribute by Strings Attached',  'A Case of You',  'spotify:track:2tjPtYQMmZyvjPTGthh8fS'],
 ['Joni Mitchell Tribute by Strings Attached',  'Blue Hotel Room',  'spotify:track:1OpzCsETUDNjY4noJRjAtP'],
 ['Joni Mitchell', 'Little Green', 'spotify:track:0HkdHu7VpgbA2NcEtZeCZd'],
 ['Joni Mitchell',  'Mitchell: Urge for Going (Instrumental Arrangement of the B-Side Track of the Joni Mitchell Single "You Turn Me on I\'m a Radio")',  'spotify:track:1I1u9aTdxxQ7SDLgBB3V7b'],
 ['Joy Division',  'Shadowplay - 2007 Remaster',  'spotify:track:4ZuC5MfGjRQs3pZtPxqMYP'],
 ['Kanye West', 'Come to Life', 'spotify:track:5xvXeuxISyXJDRbZZf4uzd'],
 ['Lady Gaga', 'Shallow', 'spotify:track:2VxeLyX666F8uXCJ0dZF8B'],
 ['Linkin Park', 'Numb', 'spotify:track:2nLtzopw4rPReszdYBJU6h'],
 ['Lord Huron',  'Harvest Moon - Recorded at Spotify Studios NYC',  'spotify:track:6gdCb962lq0Xdg7ypVLuLt'],
 ['Lord Huron',  'When the Night is Over',  'spotify:track:4FE9SgxcOoCNuGnaaU8TXz'],
 ['Louis Armstrong',  "It's Wonderful - Single Version",
  'spotify:track:7tmOKoxLQFLvQWwxzYaodT'],
 ['Sweet Sweet Loverboy',  'The Love Of My Life',  'spotify:track:4kYkySfVnGovUgY0NTNH9m'],
 ['Ludovico Einaudi',  'Einaudi: Due Tramonti',  'spotify:track:3piSeZsUx79vyAJYMpODvy'],
 ['Ludovico Einaudi', 'Experience', 'spotify:track:1BncfTJAWxrsxyT9culBrj'],
 ['Ludwig van Beethoven',  'Sonate No. 14, Op. 27/2, "Clair de Lune": I. Adagio Sostenuto',  'spotify:track:1Ee667ISk4TkLqCwf3ThIY'],
 ['Ludwig van Beethoven',  'Sonata No. 14 "Moonlight" in C-Sharp Minor", Op. 27 No. 2: I. Adagio sostenuto',  'spotify:track:3DNRdudZ2SstnDCVKFdXxG'],
 ['MXMS', 'After Night', 'spotify:track:2qGOezBrISBp7Eez493fcV'],
 ['Madeleine Peyroux',  'La Vie En Rose',  'spotify:track:1njxWb5fM5L5eEq9MyWddU'],
 ['Maria Callas',  'Norma (1997 Digital Remaster): Casta Diva',  'spotify:track:1pqGcNI8H5i3yySTNCrD7d'],
 ['Mark Knopfler',  'Romeo And Juliet - Live At Gibson Amphitheatre / June 28th 2006',  'spotify:track:6xLh2Oq3RCGSLvAHsxdvjQ'],
 ['Marvin David Levy',  'Shir Shel Moshe (Song of Moses): Kiddush',  'spotify:track:5oXQGUE9xPMU9BLQa28KNh'],
 ['Mazzy Star', 'Flowers In December', 'spotify:track:0G6Ws8Gbdt0S7pZeuYmkmm'],
 ['Metallica',  'Fade To Black (Remastered)',  'spotify:track:0dqGfCMAGyDgpUAgLNOjWd'],
 ['Wolfgang Amadeus Mozart',  'Requiem in D Minor, K. 626: III. Seq. 5, Confutatis',  'spotify:track:0HsYmC5ahRRjUOfR0cAn0j'],
 ['Wolfgang Amadeus Mozart',  'Requiem in D Minor, K. 626: III. Sequenz No. 6, Lacrimosa dies illa',  'spotify:track:4bvzJZXpkI3bkjxMCWOSu1'],
 ['My Chemical Romance',  'The Light Behind Your Eyes',  'spotify:track:3HyDpKAuR3e4l6QB7hSB2l'],
 ['Myles Kennedy',  'Year of the Tiger',  'spotify:track:1LwS1Wh33VQAsqMhw6Sl0f'],
 ['NOFX', 'Vincent', 'spotify:track:0NG094Nd0Qr6sREeFDsps0'],
 ['Neil Young',  'The Needle and The Damage Done - Live at John F. Kennedy Stadium, 13th July 1985',  'spotify:track:285vxuRky8WKTIrNP332sl'],
 ['Nick Drake', 'Pink Moon', 'spotify:track:4KROoGIaPaR1pBHPnR3bwC'],
 ['Nightwish', 'Nemo', 'spotify:track:5OF7nhjVV0dmngqYjDrZGw'],
 ['Paul McCartney',  'Here Today - Remixed 2015',  'spotify:track:0QtnwXDziZN1K55fXuLN6q'],
 ['Paul McCartney',  'I’ll Follow The Sun - Live At Amoeba 2007',  'spotify:track:3xT59EeQdq0TPGtOlXXI8t'],
 ['Paul Simon',  'Bridge Over Troubled Water',  'spotify:track:65Q3jlvZkSp55H7mlDJlAz'],
 ['Pearl Jam', 'Jeremy', 'spotify:track:62nQ8UZVqR2RMvkJHkcO2o'],
 ['Peter Gabriel',  'Sledgehammer - 2012 Remaster',  'spotify:track:4AUS8KNz7zX8XFu9L38GT0'],
 ['Peter Gabriel',  'Washing Of The Water',  'spotify:track:3GK36Qk6hy8ukjIvgiNOHJ'],
 ['Pink Floyd',  'Breathe (In the Air)',  'spotify:track:2ctvdKmETyOzPb2GiJJT53'],
 ['Pixies',  'Where Is My Mind? - Remastered',  'spotify:track:7wCmS9TTVUcIhRalDYFgPy'],
 ['Portishead', 'Roads', 'spotify:track:2sW8fmnISifQTRgnRrQTYW'],
 ['Prince', 'Manic Monday', 'spotify:track:79NYdkxwgObli2soXJovYH'],
 ['Giacomo Puccini',  "Tosca, SC 69, Act II: Vissi d'arte, vissi d'amore",
  'spotify:track:6AeYjawWTYvBvB03u0y7d4'],
 ['Queen',  'You Take My Breath Away - Remastered 2011',  'spotify:track:0JR1UPExSwwEvTRub1SZI3'],
 ['Queensrÿche',  'Scarborough Fair - Remastered/2003',  'spotify:track:0s0s823rpcq4YDFwatzdC0'],
 ['Ray LaMontagne', 'Can I Stay', 'spotify:track:6fkmApNvSEmvoVg5lReIjq'],
 ['Ray Manzarek',  'Riders on the Storm',  'spotify:track:3FvYcTXO2QtDY7kZQHku2d'],
 ['Reinbert de Leeuw',  'Gymnopédie No. 1',  'spotify:track:7LJX8Y9JagsbqnnVoB1KbO'],
 ['Sarah Brightman',  'Scarborough Fair',  'spotify:track:4vixgSEittlcLbvclr6xjN'],
 ['Sarah McLachlan', 'Blackbird', 'spotify:track:7ol4EBFZnZoxHAuUdJ6FRp'],
 ['Alexander Scriabin',  'Fantasy in B minor, Op. 28',  'spotify:track:1ChQEk7Mf5OEBXyVSOdcAj'],
 ['Simon & Garfunkel',  'Bookends Theme - Reprise',  'spotify:track:0Q3GpBYKy5Mx97TNBJVibl'],
 ['Simon & Garfunkel',  'Flowers Never Bend with the Rainfall',  'spotify:track:0qSITuCPLxjoDtESBy70WO'],
 ['Sky Cries Mary', 'Moonbathing', 'spotify:track:0R1t5owUXmMMZN9i9kzJxB'],
 ['The Smashing Pumpkins',  'Landslide - Remastered',  'spotify:track:6N7JMoaUYgo4pwPgsnSobr'],
 ['Stevie Nicks',  'Edge of Seventeen - 2016 Remaster',  'spotify:track:7L3b6iaVhDVjfo52Hbvh9Z'],
 ['Stevie Nicks',  'Leather and Lace (with Don Henley)',  'spotify:track:2npStoMbQxG1LcIeF4PZrj'],
 ['Stevie Wonder', 'Isn’t She Lovely', 'spotify:track:6wGlAaMfyhKdEPr2zycAnN'],
 ['Taylor Swift',  "All Too Well (10 Minute Version) (Taylor's Version) (From The Vault)",
  'spotify:track:5enxwA8aAbwZbf5qCHORXi'],
 ['Taylor Swift',  'the lakes - bonus track',  'spotify:track:0eFQWVz0qIxDOvhLpZ40P7'],
 ['The Karaoke Channel',  'I Can Make You a Man (Originally Performed by The Rocky Horror Picture Show (Movie Version)) [Karaoke Version]',  'spotify:track:5EvcRewjL275OXcxv3oTbn'],
 ['The Band',  'When I Paint My Masterpiece - Remastered',  'spotify:track:76WChUuOPeIK027IeUgr0l'],
 ['The Beach Boys',  'Caroline, No - Mono',  'spotify:track:5RSQKtG1KNwrzFMEePpjt6'],
 ['The Beach Boys',  'Don’t Worry Baby',  'spotify:track:6nWneWHV6S5FhXkVBeMoLE'],
 ['The Beach Boys',  "I Just Wasn't Made For These Times - Mono",
  'spotify:track:4CuO8TINNqM3D7aUdNQ3zG'],
 ['The Beach Boys',  "Let's Go Away For A While - Mono",
  'spotify:track:3GsgJI1aBrvUtqX8f3MhKT'],
 ['The Beach Boys',  'Pet Sounds - Mono',  'spotify:track:6CcgkjU9891S2S4lHKXaYh'],
 ['The Beach Boys',  'You Still Believe In Me - Mono',  'spotify:track:4fJtDPJ2NiLfFfZfu8HyYc'],
 ['Peel, David & The Apple Band',  'The wonderful world of Abbey Road',  'spotify:track:6zZU0UM3IWCGqUnEJBI8cT'],
 ['The Beatles',  'Happiness Is A Warm Gun - Remastered 2009',  'spotify:track:71LsKf3xISiOlY1mj7FFPP'],
 ['The Beatles',  'Hey Jude - Remastered 2015',  'spotify:track:0aym2LBJBk9DAYuHHutrIl'],
 ['The London Orchestral Symphony',  'I Wanna Hold Your Hand (Originally Performed by the Beatles) [Orchestral Version]',  'spotify:track:0jZYuwxjgETU8mPovGOyGf'],
 ['The Pocket Gods', 'Rubber Soul', 'spotify:track:5SdHICy96T7lqHiLhIlQDr'],
 ['The Beatles',  'Strawberry Fields Forever - Remastered 2009',  'spotify:track:3Am0IbOxmvlSXro7N5iSfZ'],
 ['The Karaoke Channel',  'Dream A Little Dream Of Me [In the Style of Beautiful South] {Karaoke Version}',  'spotify:track:3kx08p814N09kHRfA728dZ'],
 ['The Mixer Electronic Sound',  'Close To You..',  'spotify:track:0SzYkrpuA0kxcBaPx47zIs'],
 ['The Ventures', 'Superstar', 'spotify:track:6UUDZo4OIkvTNOcmgZNdLG'],
 ['The Cranberries',  'So Cold In Ireland',  'spotify:track:0x7zrNABhwPJsYMsEMaZtQ'],
 ['The Cranberries',  "When You're Gone",
  'spotify:track:2Qdnk7tbc4onLOIPd1eSNE'],
 ['The Cranberries', 'Zombie', 'spotify:track:7EZC6E7UjZe63f1jRmkWxt'],
 ['The Cure',  'A Night like This - 2006 Remaster',  'spotify:track:7cKCz7gG84i1XLvDeM3ByT'],
 ['The Cure',  'From the Edge of the Deep Green Sea',  'spotify:track:2vwBL9RVyr0vA4Og5VH0i3'],
 ['The Cure',  'In Between Days - 2006 Remaster',  'spotify:track:07CyrZF9eVd02zzIse7tZA'],
 ['The Cure', 'A Letter to Elise', 'spotify:track:4DdXOLc1VMAY34ourCn1Xa'],
 ['The Cure',  'Lullaby - 2010 Remaster',  'spotify:track:4d4oXk7O2lEhZ83ivV93li'],
 ['The Cure', 'The Lovecats', 'spotify:track:5rzr3lYGJiw3OXEg8imWgH'],
 ['The Cure',  'Untitled - 2010 Remaster',  'spotify:track:3KwSkeKjuTRSwcN5cGp2ym'],
 ['The Doors', 'L.A. Woman', 'spotify:track:6DmfWj5kOa1fX8AwN9byOn'],
 ['The Eagles', 'The Desperadoes', 'spotify:track:10ppF835WJMYI5v65gFLZ3'],
 ['The Illegal Eagles',  'Seven Bridges Road',  'spotify:track:02fH7IGrphx5WMoQIifiju'],
 ['The Killers', "Sam's Town", 'spotify:track:77Kh98jRHI40rIAKnBs7cl'],
 ['The Symphonic Rock All-Stars',  'Bittersweet Symphony (Made Famous by The Verve)',  'spotify:track:6B8Yu77H2FCTVudOeGLF7K'],
 ['The War On Drugs', 'Pain', 'spotify:track:59P1nrdEImkAKa1nyW9X2e'],
 ['TOOL', 'Intension', 'spotify:track:2ae6mkuD2gJnJQADl488et'],
 ['Traditional', 'Amazing Grace', 'spotify:track:4AOZxTd1CuUDIWRYJrvnDL'],
 ['U2', 'One', 'spotify:track:3G69vJMWsX6ZohTykad2AU'],
 ['Van Morrison',  'Madame George - 1999 Remaster',  'spotify:track:1N4MKISvC1ddfRCRQDXDd2'],
 ['Various Artists',  'Down to the River to Pray (From "O Brother, Where Art Thou?")',  'spotify:track:2ysPUG3ZlrAdJ4iNmueTcK'],
 ['machineheart', 'Stonecold', 'spotify:track:5SmYtofSMKeccnqWksS9vW']]


In [62]:
a1 = [['ABBA', 'Dancing Queen', 'spotify:track:0GjEhVFGZW8afUYGChu3Rr'],
 ['Alice In Chains', 'I Stay Away', 'spotify:track:6HnZsLUDrG3HTSbh8sq2u3'],
 ['Alison Krauss', 'Baby Mine', 'spotify:track:7GJ8aXAGdfjo2OAh6XreB6'],
]
addlist2 = [a[2] for a in a1]
print (len(addlist2), 'items')


3 items


In [63]:
while(addlist2):
    sp.user_playlist_add_tracks(os.getenv('SPOTIFY_USERNAME'), 
                                playlist_id=playlist_id, 
                                tracks=addlist2[-100:])
    addlist2 = addlist2[:-100]
    print("added items, remaining ", len(addlist2))


added items, remaining  0


In [ ]:
len(res3[474].body)

In [ ]:
test = res3[474].body
test[:1000]

In [ ]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chat_models import ChatOpenAI

qa_model = 'gpt-3.5-turbo-0301'

llm = ChatOpenAI(model_name=qa_model, 
                 temperature=0.2,
                )

# llm = OpenAI(temperature=0.9,
#              model_name="text-davinci-003"
#             )

text = "What would be a good company name for a company that makes colorful socks?"

conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

conversation.predict(input="Hi there!")

print(llm(text))

In [ ]:

template = """"Define an example CSV file output as follows: 
"artist","song_title"
"The Beatles","Yesterday"
"Eagles","Hotel California"

You will extract all song titles and artists from the input, and return a CSV file output of the artists and song titles you extract from the input. If there were no songs extracted from the input, return "no songs found". the input is:

{question}
"""

prompt = PromptTemplate(
        template=template,
    input_variables=['question']
)

# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm
)

# ask the question 
print(llm_chain.run(res3[big_ones[0]].body))



In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

system_prompt=""
template="""You will act as a research assistant finding all the artists and track names in a document and returning them in a comma separated values format.
Extract all track names and artists from the input below.
Your response should be a list of comma separated values containing the artists and track names you extract from the input. 
The header row should contain `"artist","track_name"`. 
{format_instructions}
The fields of each record should be enclosed in double-quotes. The input is:"
{input_text}
"""
format_instructions='Your response should be a list of comma separated values, eg: `"artist", "track_name"`'

output_parser = CommaSeparatedListOutputParser()
prompt = PromptTemplate(
    template=template,
    input_variables=["input_text"],
    partial_variables={"format_instructions": format_instructions}
)

_input = prompt.format(input_text=res3[big_ones[0]].body)
model = OpenAI(temperature=0)
output = model(_input)


In [ ]:
from pydantic import BaseModel, Field, validator


class ArtistTrack(BaseModel):
    artist: str = Field(description="name of an artist")
    track_name: str = Field(description="name of a track")
        

parser = PydanticOutputParser(pydantic_object=ArtistTrack)

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI(temperature=0))

In [ ]:
new_parser.parse(output)


In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

model_name = 'text-davinci-003'
temperature = 0.0
model = ChatOpenAI(temperature=temperature)

# a query to prompt a language model to populate the data structure.
prompt_template = """
You will act as a research assistant finding all the artists and track names in a document.
You will extract all track names and artists from the input below.
Your response should be a list of records containing artist and track_name extracted from the input.
{format_instructions}
The input is:
{query}"""

# Define data structure.        
class ArtistTrack(BaseModel):
    artist: str = Field(description="name of an artist")
    track_name: str = Field(description="name of a track")
        
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=ArtistTrack)

# Cleanup parser can fix format if it doesn't work first time
cleanup_parser = OutputFixingParser.from_llm(parser=parser, llm=model)    

# define a chain using prompt template
llm_chain = LLMChain(
    prompt=PromptTemplate(
        template=prompt_template,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    ),
    llm=model
)

output = llm_chain.run(test)
        
for s in output.split("\n"):
    z = cleanup_parser.parse(s)
    print(str(z))


In [ ]:
output

In [ ]:
for s in output.split("\n"):
    z = parser.parse(s)
    print(str(z))

In [ ]:
z

In [ ]:
from langchain.output_parsers import RetryWithErrorOutputParser
retry_parser = RetryWithErrorOutputParser.from_llm(parser=parser, llm=OpenAI(temperature=0))
prompt_value = prompt.format_prompt(query=test)
retry_parser.parse_with_prompt(output, prompt_value)


In [ ]:
output = '[%s]' % output
print(output)
